In [1]:
import torch

In [2]:
def calc_loss(y_true, y_pred):
    # 1. 取出真实的标签
    # y_true = y_true[::2]    # tensor([1, 0, 1]) 真实的标签

    # 2. 对输出的句子向量进行l2归一化   后面只需要对应为相乘  就可以得到cos值了
    norms = (y_pred ** 2).sum(axis=1, keepdims=True) ** 0.5
    # y_pred = y_pred / torch.clip(norms, 1e-8, torch.inf)
    y_pred = y_pred / norms

    # 3. 奇偶向量相乘
    y_pred = torch.sum(y_pred[::2] * y_pred[1::2], dim=1) * 20

    # 4. 取出负例-正例的差值
    y_pred = y_pred[:, None] - y_pred[None, :]  # 这里是算出所有位置 两两之间余弦的差值
    # 矩阵中的第i行j列  表示的是第i个余弦值-第j个余弦值
    y_true = y_true[:, None] < y_true[None, :]   # 取出负例-正例的差值
    y_true = y_true.float()
    y_pred = y_pred - (1 - y_true) * 1e12
    y_pred = y_pred.view(-1)
        
    if torch.cuda.is_available():
        y_pred = torch.cat((torch.tensor([0]).float().cuda(), y_pred), dim=0)  # 这里加0是因为e^0 = 1相当于在log中加了1
    else:
        y_pred = torch.cat((torch.tensor([0]).float(), y_pred), dim=0)  # 这里加0是因为e^0 = 1相当于在log中加了1
        
    return torch.logsumexp(y_pred, dim=0)

In [3]:
device = 'cuda'
y_true = torch.tensor([0])
# y_pred = torch.tensor([0, 1, 0, 1, 0, 1])
y_pred = torch.tensor([[0.0065, 0.0138]])

y_true = y_true.to(device)
y_pred = y_pred.to(device)

In [4]:
calc_loss(y_true, y_pred) 

y_true tensor([[0.]], device='cuda:0')
y_pred tensor([], device='cuda:0')


tensor(0., device='cuda:0')